# Encontro 4: Memória de Curto e Longo Prazo

Neste encontro vamos implementar e entender diferentes tipos de memória em aplicações com LangChain: curto e longo prazo, janelas de contexto e armazenamento externo. Também veremos como integrar a memória ao runtime do agente. O tom é didático, com exemplos práticos.

Assuntos:
- Implementando memória de curto e longo prazo
- Tipos de memória (buffer, janela, sumarização, armazenamento)
- Gerenciamento de janelas de contexto
- Armazenando histórico de conversas em bancos externos
- Integrando a memória ao runtime do agente


In [ ]:
%pip install -q -r ../requirements.txt

In [2]:
import os
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI

# Inicialização do LLM
load_dotenv()
api_key = os.getenv("GOOGLE_API_KEY")
model_name = os.getenv("MODEL_NAME", "gemini-2.0-flash")
assert api_key, "GOOGLE_API_KEY ausente. Defina no .env."
llm = ChatGoogleGenerativeAI(model=model_name, google_api_key=api_key, streaming=False)
print("LLM pronto.")


LLM pronto.


## 1) Tipos de memória

- Buffer de conversas: guarda todas as mensagens (curto prazo simples, mas cresce indefinidamente).
- Janela de contexto: mantém apenas as últimas *k* mensagens (controle de tokens, foco no recente).
- Sumarização: comprime o histórico com resumos iterativos (longa duração com menor custo).
- Armazenamento externo: persiste histórico em DBs (SQLite/Postgres), útil para múltiplas sessões.

Veremos como aplicar cada uma com LCEL e, em seguida, integrar ao runtime do agente.


In [4]:
# Memória básica com LCEL: ChatMessageHistory + RunnableWithMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableWithMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.output_parsers import StrOutputParser

store = {}  # dicionário simples {session_id: InMemoryChatMessageHistory}

def get_session_history(session_id: str):
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

prompt = ChatPromptTemplate.from_messages([
    ("system", "Você é um assistente. Mantenha contexto da conversa."),
    MessagesPlaceholder("chat_history"),
    ("human", "{input}"),
])
chain = prompt | llm | StrOutputParser()
chain_with_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

print("Sessão A")
out1 = chain_with_history.invoke({"input": "Oi, quem é você?"}, config={"configurable": {"session_id": "A"}})
out2 = chain_with_history.invoke({"input": "E o que você lembra do que falei?"}, config={"configurable": {"session_id": "A"}})
print(out1)
print(out2)

print("Sessão B")
out3 = chain_with_history.invoke({"input": "Olá!"}, config={"configurable": {"session_id": "B"}})
print(out3)


Sessão A
Eu sou um modelo de linguagem grande, treinado pelo Google.
Eu lembro que você me perguntou quem eu sou e eu te respondi que sou um modelo de linguagem grande, treinado pelo Google.
Sessão B
Olá! Como posso te ajudar hoje?



## 2) Gerenciamento de janelas de contexto

A ideia é limitar o histórico para reduzir custo de tokens e manter foco no recente. Há duas abordagens:
- Regras próprias (manter apenas as últimas *k* mensagens).
- `ConversationBufferWindowMemory` pronto (API clássica do LangChain).


In [5]:
# Exemplo simples: armazenar apenas as últimas k mensagens por sessão
from collections import defaultdict
from langchain_core.chat_history import InMemoryChatMessageHistory

store_k = {}
def get_session_history_k(session_id: str, k: int = 4):
    hist = store_k.get(session_id)
    if hist is None:
        hist = InMemoryChatMessageHistory()
        store_k[session_id] = hist
    # Enxuga a história mantendo só as últimas k mensagens
    if len(hist.messages) > k:
        hist.messages = hist.messages[-k:]
    return hist

chain_window = RunnableWithMessageHistory(
    chain,
    lambda sid: get_session_history_k(sid, k=4),
    input_messages_key="input",
    history_messages_key="chat_history",
)
out_w1 = chain_window.invoke({"input": "Contexto 1"}, config={"configurable": {"session_id": "W"}})
out_w2 = chain_window.invoke({"input": "Contexto 2"}, config={"configurable": {"session_id": "W"}})
out_w3 = chain_window.invoke({"input": "Contexto 3"}, config={"configurable": {"session_id": "W"}})
out_w4 = chain_window.invoke({"input": "Contexto 4"}, config={"configurable": {"session_id": "W"}})
out_w5 = chain_window.invoke({"input": "Agora, o que lembra?"}, config={"configurable": {"session_id": "W"}})
print(out_w5)


No momento, lembro que estamos trabalhando com os seguintes contextos:

*   Contexto 1
*   Contexto 2
*   Contexto 3
*   Contexto 4

Por favor, me diga como posso te ajudar com base nesses contextos.



In [6]:
# API clássica: ConversationBufferWindowMemory
from langchain.memory import ConversationBufferWindowMemory
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

mem_win = ConversationBufferWindowMemory(k=3, return_messages=True)
tmpl = PromptTemplate.from_template("{input}")
chain_classic = LLMChain(llm=llm, prompt=tmpl, memory=mem_win, verbose=False)
chain_classic.run("Oi!")
chain_classic.run("Estou configurando uma janela de contexto.")
resp = chain_classic.run("O que você lembra agora?")
print(resp)


C:\Users\anton\AppData\Local\Temp\ipykernel_19096\3086726850.py:6: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  mem_win = ConversationBufferWindowMemory(k=3, return_messages=True)
C:\Users\anton\AppData\Local\Temp\ipykernel_19096\3086726850.py:8: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain_classic = LLMChain(llm=llm, prompt=tmpl, memory=mem_win, verbose=False)
C:\Users\anton\AppData\Local\Temp\ipykernel_19096\3086726850.py:9: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chain_classic.run("Oi!")


Como um modelo de linguagem grande, eu não tenho memórias no sentido humano. Eu não "lembro" de coisas da mesma forma que você. 

O que eu posso te dizer é que, neste momento, estou ativo e disponível para responder às suas perguntas. Posso acessar e processar informações do mundo real através da Pesquisa Google e fornecer respostas relevantes e abrangentes.

Em termos práticos, o que "lembro" agora é:

*   **Que estou conversando com você:** Sei que sou um modelo de linguagem e que estou interagindo com um usuário.
*   **O contexto da nossa conversa:** Mantenho um histórico da nossa conversa atual para entender o que você está perguntando e responder de forma coerente.
*   **Informações sobre o meu treinamento:** Tenho conhecimento sobre o vasto conjunto de dados textuais e de código com o qual fui treinado.

Se você tem alguma pergunta específica, pode perguntar! Estou aqui para ajudar.



## 3) Memória de longo prazo (sumarização)

Para conversas muito longas, sumarizar o histórico reduz tokens.
`ConversationSummaryMemory` usa o LLM para gerar resumos iterativos.


In [7]:
from langchain.memory import ConversationSummaryMemory
from langchain.chains import ConversationChain

summary_mem = ConversationSummaryMemory(llm=llm, return_messages=True)
conv_chain = ConversationChain(llm=llm, memory=summary_mem, verbose=False)
conv_chain.run("Vamos discutir um plano em 5 etapas.")
conv_chain.run("Etapa 1: levantar requisitos.")
conv_chain.run("Etapa 2: desenhar arquitetura.")
resp = conv_chain.run("Resuma o que foi discutido até agora.")
print(resp)


C:\Users\anton\AppData\Local\Temp\ipykernel_19096\3988066545.py:4: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  summary_mem = ConversationSummaryMemory(llm=llm, return_messages=True)
C:\Users\anton\AppData\Local\Temp\ipykernel_19096\3988066545.py:5: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :meth:`~RunnableWithMessageHistory: https://python.langchain.com/v0.2/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html` instead.
  conv_chain = ConversationChain(llm=llm, memory=summary_mem, verbose=False)
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api

ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. 
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 15, model: gemini-2.0-flash
Please retry in 53.348628436s. [links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerMinutePerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-2.0-flash"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 15
}
, retry_delay {
  seconds: 53
}
]

## 4) Armazenando histórico em banco externo (SQLite)

Com `SQLChatMessageHistory`, podemos persistir mensagens por sessão em um banco SQL.
Usaremos SQLite por simplicidade.


In [8]:
%pip install -q sqlalchemy

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
from langchain_community.chat_message_histories import SQLChatMessageHistory
from langchain_core.runnables import RunnableWithMessageHistory

def get_sql_history(session_id: str):
    # Usa SQLite em arquivo local
    return SQLChatMessageHistory(session_id=session_id, connection_string="sqlite:///encontro4_memory.db")

chain_sql = RunnableWithMessageHistory(
    chain,
    get_sql_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)
resp_sql1 = chain_sql.invoke({"input": "Olá, quero registrar no DB externo."}, config={"configurable": {"session_id": "DB1"}})
resp_sql2 = chain_sql.invoke({"input": "O que você lembra desta sessão?"}, config={"configurable": {"session_id": "DB1"}})
print(resp_sql1)
print(resp_sql2)


c:\Users\anton\AppData\Local\Programs\Python\Python312\Lib\site-packages\langchain_core\runnables\history.py:608: LangChainDeprecationWarning: `connection_string` was deprecated in LangChain 0.2.2 and will be removed in 1.0. Use connection instead.
  message_history = self.get_session_history(


Olá! Para te ajudar com o registro no banco de dados externo, preciso de mais algumas informações:

1.  **Qual tipo de banco de dados você está usando?** (Ex: MySQL, PostgreSQL, MongoDB, etc.)
2.  **Qual linguagem de programação ou ferramenta você está utilizando para interagir com o banco de dados?** (Ex: Python, Node.js, PHP, etc.)
3.  **Você já tem as credenciais de acesso ao banco de dados?** (host, usuário, senha, nome do banco)
4.  **Quais dados você quer registrar?** (nomes dos campos e seus respectivos tipos)
5.  **Qual é o contexto desse registro?** (Ex: registro de um novo usuário, envio de dados de um formulário, etc.)

Com essas informações, posso te dar um exemplo de código ou te guiar no processo de registro.
Lembro que você quer registrar informações em um banco de dados externo e estou aguardando você me fornecer detalhes sobre:

*   O tipo de banco de dados que você está usando.
*   A linguagem de programação ou ferramenta que você está utilizando.
*   As credenciais d

## 5) Integrando a memória ao runtime do agente

Vamos criar um agente de ferramentas com `chat_history` no prompt e usar `RunnableWithMessageHistory` para manter a memória por sessão.


In [10]:
from typing import List
from pydantic import BaseModel, Field
from langchain.tools import StructuredTool
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables import RunnableWithMessageHistory

class CalculaMediaArgs(BaseModel):
    numeros: List[float] = Field(..., description="Lista de números para média")

def calcula_media(numeros: List[float]) -> str:
    if not numeros:
        return "Lista vazia; informe ao menos um número."
    media = sum(numeros) / len(numeros)
    return f"Média: {media:.4f}"

calcula_media_tool = StructuredTool.from_function(
    name="calcula_media",
    description="Calcula a média aritmética.",
    func=calcula_media,
    args_schema=CalculaMediaArgs,
)

tools = [calcula_media_tool]
prompt_agent = ChatPromptTemplate.from_messages([
    ("system", "Você é um assistente. Use ferramentas quando necessário."),
    MessagesPlaceholder("chat_history"),
    ("human", "{input}"),
    MessagesPlaceholder("agent_scratchpad"),
])
agent = create_tool_calling_agent(llm, tools, prompt_agent)
executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# Memória por sessão para o agente
agent_store = {}
def get_agent_history(session_id: str):
    if session_id not in agent_store:
        agent_store[session_id] = InMemoryChatMessageHistory()
    return agent_store[session_id]

agent_with_memory = RunnableWithMessageHistory(
    executor,
    get_agent_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

sid = "AG1"
r1 = agent_with_memory.invoke({"input": "Olá!"}, config={"configurable": {"session_id": sid}})
r2 = agent_with_memory.invoke({"input": "Calcule a média de 2, 4 e 6."}, config={"configurable": {"session_id": sid}})
r3 = agent_with_memory.invoke({"input": "O que você lembra dessa conversa?"}, config={"configurable": {"session_id": sid}})
print(r1.get("output") or r1)
print(r2.get("output") or r2)
print(r3.get("output") or r3)




> Entering new AgentExecutor chain...
{'name': 'calcula_media', 'description': 'Calcula a média aritmética.', 'parameters': {'properties': {'numeros': {'description': 'Lista de números para média', 'items': {'type': 'number'}, 'type': 'array'}}, 'required': ['numeros'], 'type': 'object'}}
Olá! Como posso ajudar?

> Finished chain.


> Entering new AgentExecutor chain...
{'name': 'calcula_media', 'description': 'Calcula a média aritmética.', 'parameters': {'properties': {'numeros': {'description': 'Lista de números para média', 'items': {'type': 'number'}, 'type': 'array'}}, 'required': ['numeros'], 'type': 'object'}}

Invoking: `calcula_media` with `{'numeros': [2.0, 4.0, 6.0]}`


Média: 4.0000{'name': 'calcula_media', 'description': 'Calcula a média aritmética.', 'parameters': {'properties': {'numeros': {'description': 'Lista de números para média', 'items': {'type': 'number'}, 'type': 'array'}}, 'required': ['numeros'], 'type': 'object'}}
A média de 2, 4 e 6 é 4.

> Finished chain.

## 6) Boas práticas

- Limite o histórico com janelas ou resumos para controlar custo.
- Prefira sumarização quando o contexto for muito longo.
- Use armazenamento externo para sessões persistentes ou multiusuário.
- Documente chaves (`input`, `chat_history`) e como o runtime injeta a memória.
- Teste: verifique se o que o agente "lembra" corresponde ao que foi armazenado.
